# Task 1: Automatic classification of tweets that report adverse effects

In [1]:
!pip install -q transformers contractions imbalanced-learn ekphrasis

     |████████████████████████████████| 4.0 MB 13.3 MB/s 
     |████████████████████████████████| 80 kB 10.5 MB/s 
     |████████████████████████████████| 6.6 MB 65.0 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 68.1 MB/s 
     |████████████████████████████████| 895 kB 74.5 MB/s 
     |████████████████████████████████| 106 kB 67.8 MB/s 
     |████████████████████████████████| 287 kB 72.7 MB/s 
     |████████████████████████████████| 45 kB 3.5 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 


## 1. Import all the necessary libraries and data files

In [2]:
import numpy as np
import pandas as pd

import warnings
import torch
import torch.nn as nn
import time

from sklearn.metrics import classification_report
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertTokenizerFast
from transformers import RobertaTokenizerFast, RobertaModel
from transformers import AutoTokenizer, AutoModel
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from tqdm import tqdm
from sklearn.metrics import f1_score

warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=None

In [3]:
train_filename = "task2_en_training.tsv"
val_filename = "task2_en_validation.tsv"

In [4]:
# Load data
train = pd.read_csv(train_filename, sep="\t")
validation = pd.read_csv(val_filename, sep="\t")

In [5]:
print(f"Shape of training data is {train.shape} and validation data is {validation.shape}")

Shape of training data is (20544, 4) and validation data is (5134, 4)


In [6]:
# Train top 5 rows
train.head().style.set_caption("Task 1: Train dataset")

,tweet_id,user_id,class,tweet
0,344266386467606528,809439366,0,"depression hurts, cymbalta can help"
1,349220537903489025,323112996,0,"@jessicama20045 right, but cipro can make things much worse...and why give bayer more of your money? they already screwed you once w/ essure"
2,351421773079781378,713100330,0,@fibby1123 are you on paxil .. i need help
3,326594278472171520,543113070,0,@redicine the lamotrigine and sjs just made chaos more vengeful and sadistic.
4,345567138376994816,138795534,0,"have decided to skip my #humira shot today. my body's having hysterics, need time to simmer down #rheum"


## 2. Prepare the data - Clean & Prepare for Model

In [7]:
# Drop unwanted columns
train.drop(['tweet_id', 'user_id'], axis=1, inplace=True)
validation.drop(['tweet_id', 'user_id'], axis=1, inplace=True)

In [8]:
# Referred from: https://github.com/cbaziotis/ekphrasis

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


In [9]:
train['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in train.tweet]
validation['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in validation.tweet]

In [10]:
# Train top 5 rows after pre-processing
train[['class', 'clean_tweets']].head()

,class,clean_tweets
0,0,"depression hurts , cymbalta can help"
1,0,"<user> right , but cipro can make things much worse . <repeated> and why give bayer more of your money ? they already screwed you once w / essure"
2,0,<user> are you on paxil . <repeated> i need help
3,0,<user> the lamotrigine and sjs just made chaos more vengeful and sadistic .
4,0,"have decided to skip my <hashtag> humira </hashtag> shot today . my body ' s having hysterics , need time to simmer down <hashtag> rheum </hashtag>"


In [11]:
BATCH_SIZE = 32
N_EPOCHS = 5

In [12]:
# Define BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

In [13]:
# Tokenize train and validation data
train_enc_sci = tokenizer.batch_encode_plus(train.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")
valid_enc_sci = tokenizer.batch_encode_plus(validation.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")

In [14]:
train_enc_sci.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [15]:
train_enc_sci.input_ids.shape, train_enc_sci.token_type_ids.shape, train_enc_sci.attention_mask.shape

(torch.Size([20544, 128]), torch.Size([20544, 128]), torch.Size([20544, 128]))

In [16]:
BATCH_SIZE = 32
N_EPOCHS = 5

In [17]:
def get_dataloader(encoding, target):
    data = (TensorDataset(encoding.input_ids, encoding.token_type_ids, encoding.attention_mask, target))
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader

In [18]:
train_dataloader_sci = get_dataloader(train_enc_sci, torch.tensor(train['class'].to_list()))
valid_dataloader_sci = get_dataloader(valid_enc_sci, torch.tensor(validation['class'].to_list()))

In [19]:
# Sanity check that the tensors returned by the dataloader are correct
for batch in train_dataloader_sci:
    input_ids, type_ids, attn_mask, target = batch
    print(input_ids.shape, type_ids.shape, attn_mask.shape, target.shape)
    break

torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32])


## 3. Model Building - SciBERT

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [21]:
class SciBERTclassifier(nn.Module):
    def __init__(self, transformer):
        super(SciBERTclassifier, self).__init__()
        self.transformer = transformer
        self.linear_layer = nn.Linear(768, 2)
    
    def forward(self, ip_ids, type_ids, attn_mask):
        op = self.transformer(input_ids=ip_ids,
                              attention_mask=attn_mask, 
                              token_type_ids=type_ids)
        return  self.linear_layer(op["pooler_output"])

In [22]:
def count_parameter(model):
    return sum(para.numel() for para in model.parameters() if para.requires_grad)

In [23]:
transformer = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
model = SciBERTclassifier(transformer).to(device)
print(f"The model has {count_parameter(model):,} trainable parameters.")

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The model has 109,920,002 trainable parameters.


In [24]:
# for name, param in model.named_parameters():
#     if "pooler" in name or "linear" in name:#or "layer.11" in name or "layer.10" in name or "linear" in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False
#   print(name, param.shape, param.requires_grad)

print(f'The model has {count_parameter(model):,} trainable parameters')

The model has 109,920,002 trainable parameters


In [25]:
# Define optimizer and 
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.AdamW(model.parameters(), lr = 2e-5)

#### Train SciBERT Model

In [26]:
def train_model(model, dataloader, clip=1.0):
    model.train()

    epoch_loss = 0
    batch_num = 0
    pred, target = [], []

    for index, batch in tqdm(enumerate(dataloader)):
        batch = tuple(row.to(device) for row in batch)
        input_ids, type_ids, attn_mask, y = batch

        optim.zero_grad()
        output = model(input_ids, type_ids, attn_mask)
        loss = criterion(output, y)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optim.step()

        epoch_loss += loss.item()
        batch_num += 1
        pred.extend(torch.argmax(output, -1).tolist())
        target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred)

def evaluate(model, dataloader):
    model.eval()

    epoch_loss = 0
    batch_num = 0
    pred, target = list(), list()

    for index, batch in enumerate(dataloader):
        batch = tuple(row.to(device) for row in batch)
        input_ids, type_ids, attn_mask, y = batch
        
        with torch.no_grad():
            output = model(input_ids, type_ids, attn_mask)
            loss = criterion(output, y)
            
            epoch_loss += loss.item()
            batch_num += 1
            pred.extend(torch.argmax(output, -1).tolist())
            target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred), pred, target

In [27]:
best_valid_loss = float('inf')
total_train_loss, total_valid_loss = list(), list()

In [28]:
for epoch in tqdm(range(N_EPOCHS)):
    train_loss, train_f1_score = train_model(model, train_dataloader_sci)
    total_train_loss.append(train_loss)

    valid_loss, valid_f1_score, pred, target = evaluate(model, valid_dataloader_sci)
    total_valid_loss.append(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_pred, best_target = pred, target
        torch.save(model.state_dict(), "model_least_loss.pt")
        print("\nBest Model Saved!!\n")
    
    # elif epoch % 3 == 0:
    torch.save(model.state_dict(), "model_checkpoint_sci" + str(epoch+1) + ".pt")
    print("\nCheckpoint Model Saved!\n")
    
    print(f"Epoch: {epoch+1:02}")
    print(f"Train Total Loss: {train_loss:.3f} | Train F1 Score: {train_f1_score:.3f}")
    print(f"Valid Total Loss: {valid_loss:.3f} | Valid F1 Score: {valid_f1_score:.3f}")
    print("-"*20)

  0%|          | 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  1.18it/s]
2it [00:01,  1.43it/s]
3it [00:02,  1.53it/s]
4it [00:02,  1.59it/s]
5it [00:03,  1.62it/s]
6it [00:03,  1.63it/s]
7it [00:04,  1.64it/s]
8it [00:05,  1.65it/s]
9it [00:05,  1.66it/s]
10it [00:06,  1.66it/s]
11it [00:06,  1.66it/s]
12it [00:07,  1.66it/s]
13it [00:08,  1.67it/s]
14it [00:08,  1.66it/s]
15it [00:09,  1.66it/s]
16it [00:09,  1.66it/s]
17it [00:10,  1.66it/s]
18it [00:11,  1.67it/s]
19it [00:11,  1.66it/s]
20it [00:12,  1.65it/s]
21it [00:12,  1.65it/s]
22it [00:13,  1.61it/s]
23it [00:14,  1.60it/s]
24it [00:14,  1.60it/s]
25it [00:15,  1.61it/s]
26it [00:16,  1.60it/s]
27it [00:16,  1.61it/s]
28it [00:17,  1.56it/s]
29it [00:17,  1.57it/s]
30it [00:18,  1.58it/s]
31it [00:19,  1.59it/s]
32it [00:19,  1.60it/s]
33it [00:20,  1.57it/s]
34it [00:21,  1.57it/s]
35it [00:21,  1.56it/s]
36it [00:22,  1.57it/s]
37it [00:23,  1.58it/s]
38it [00:23,  1.57it/s]
39it [00:24,  1.57it/s]
40it [00:24,  1.


Best Model Saved!!



 20%|██        | 1/5 [07:47<31:09, 467.36s/it]


Checkpoint Model Saved!

Epoch: 01
Train Total Loss: 0.223 | Train F1 Score: 0.365
Valid Total Loss: 0.192 | Valid F1 Score: 0.551
--------------------



0it [00:00, ?it/s]
1it [00:00,  1.40it/s]
2it [00:01,  1.48it/s]
3it [00:02,  1.48it/s]
4it [00:02,  1.49it/s]
5it [00:03,  1.49it/s]
6it [00:04,  1.49it/s]
7it [00:04,  1.49it/s]
8it [00:05,  1.49it/s]
9it [00:06,  1.49it/s]
10it [00:06,  1.49it/s]
11it [00:07,  1.50it/s]
12it [00:08,  1.49it/s]
13it [00:08,  1.49it/s]
14it [00:09,  1.49it/s]
15it [00:10,  1.49it/s]
16it [00:10,  1.49it/s]
17it [00:11,  1.49it/s]
18it [00:12,  1.49it/s]
19it [00:12,  1.49it/s]
20it [00:13,  1.49it/s]
21it [00:14,  1.49it/s]
22it [00:14,  1.49it/s]
23it [00:15,  1.49it/s]
24it [00:16,  1.49it/s]
25it [00:16,  1.49it/s]
26it [00:17,  1.49it/s]
27it [00:18,  1.48it/s]
28it [00:18,  1.49it/s]
29it [00:19,  1.49it/s]
30it [00:20,  1.49it/s]
31it [00:20,  1.48it/s]
32it [00:21,  1.48it/s]
33it [00:22,  1.48it/s]
34it [00:22,  1.48it/s]
35it [00:23,  1.48it/s]
36it [00:24,  1.48it/s]
37it [00:24,  1.48it/s]
38it [00:25,  1.48it/s]
39it [00:26,  1.48it/s]
40it [00:26,  1.47it/s]
41it [00:27,  1.48it/s]
42it 

...


Checkpoint Model Saved!

Epoch: 03
Train Total Loss: 0.092 | Train F1 Score: 0.825
Valid Total Loss: 0.243 | Valid F1 Score: 0.557
--------------------



0it [00:00, ?it/s]
1it [00:00,  1.53it/s]
2it [00:01,  1.49it/s]
3it [00:02,  1.49it/s]
4it [00:02,  1.49it/s]
5it [00:03,  1.49it/s]
6it [00:04,  1.49it/s]
7it [00:04,  1.50it/s]
8it [00:05,  1.49it/s]
9it [00:06,  1.49it/s]
10it [00:06,  1.49it/s]
11it [00:07,  1.49it/s]
12it [00:08,  1.50it/s]
13it [00:08,  1.49it/s]
14it [00:09,  1.50it/s]
15it [00:10,  1.50it/s]
16it [00:10,  1.49it/s]
17it [00:11,  1.49it/s]
18it [00:12,  1.49it/s]
19it [00:12,  1.49it/s]
20it [00:13,  1.49it/s]
21it [00:14,  1.50it/s]
22it [00:14,  1.49it/s]
23it [00:15,  1.50it/s]
24it [00:16,  1.49it/s]
25it [00:16,  1.49it/s]
26it [00:17,  1.49it/s]
27it [00:18,  1.49it/s]
28it [00:18,  1.50it/s]
29it [00:19,  1.50it/s]
30it [00:20,  1.49it/s]
31it [00:20,  1.49it/s]
32it [00:21,  1.49it/s]
33it [00:22,  1.49it/s]
34it [00:22,  1.49it/s]
35it [00:23,  1.49it/s]
36it [00:24,  1.49it/s]
37it [00:24,  1.49it/s]
38it [00:25,  1.49it/s]
39it [00:26,  1.48it/s]
40it [00:26,  1.48it/s]
41it [00:27,  1.49it/s]
42it 


Checkpoint Model Saved!

Epoch: 04
Train Total Loss: 0.054 | Train F1 Score: 0.910
Valid Total Loss: 0.350 | Valid F1 Score: 0.550
--------------------



0it [00:00, ?it/s]
1it [00:00,  1.54it/s]
2it [00:01,  1.50it/s]
3it [00:02,  1.49it/s]
4it [00:02,  1.49it/s]
5it [00:03,  1.49it/s]
6it [00:04,  1.49it/s]
7it [00:04,  1.49it/s]
8it [00:05,  1.49it/s]
9it [00:06,  1.49it/s]
10it [00:06,  1.49it/s]
11it [00:07,  1.49it/s]
12it [00:08,  1.49it/s]
13it [00:08,  1.49it/s]
14it [00:09,  1.49it/s]
15it [00:10,  1.49it/s]
16it [00:10,  1.49it/s]
17it [00:11,  1.49it/s]
18it [00:12,  1.49it/s]
19it [00:12,  1.49it/s]
20it [00:13,  1.49it/s]
21it [00:14,  1.49it/s]
22it [00:14,  1.49it/s]
23it [00:15,  1.49it/s]
24it [00:16,  1.49it/s]
25it [00:16,  1.49it/s]
26it [00:17,  1.50it/s]
27it [00:18,  1.50it/s]
28it [00:18,  1.50it/s]
29it [00:19,  1.50it/s]
30it [00:20,  1.50it/s]
31it [00:20,  1.49it/s]
32it [00:21,  1.49it/s]
33it [00:22,  1.49it/s]
34it [00:22,  1.49it/s]
35it [00:23,  1.49it/s]
36it [00:24,  1.49it/s]
37it [00:24,  1.49it/s]
38it [00:25,  1.49it/s]
39it [00:26,  1.49it/s]
40it [00:26,  1.49it/s]
41it [00:27,  1.49it/s]
42it 


Checkpoint Model Saved!

Epoch: 05
Train Total Loss: 0.033 | Train F1 Score: 0.951
Valid Total Loss: 0.412 | Valid F1 Score: 0.582
--------------------


In [29]:
print(classification_report(best_target, best_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      4660
           1       0.64      0.48      0.55       474

    accuracy                           0.93      5134
   macro avg       0.80      0.73      0.76      5134
weighted avg       0.92      0.93      0.92      5134



In [35]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [30]:
from glob import glob

In [31]:
for filepath in glob("*.pt"):
    print(filepath)

model_checkpoint_sci3.pt
model_checkpoint_sci4.pt
model_checkpoint_sci1.pt
model_checkpoint_sci5.pt
model_least_loss.pt
model_checkpoint_sci2.pt


In [36]:
!cp -r model_least_loss.pt /content/gdrive/My\ Drive/Colab\ Notebooks/

In [37]:
# for filepath in glob("*.pt"):
#     !cp -r $filepath /content/gdrive/My\ Drive/Colab\ Notebooks/
#     time.sleep(10)